<a href="https://colab.research.google.com/github/mgnarag/binarization-lowres/blob/main/Metric_RGB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [37]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [38]:
!ls drive/My\ Drive
file_path = "/content/drive/My Drive/Architectural_designs/From Arki/"

 313490484_646790150357239_663796780442369775_n.jpg
 361079911_248325231393045_2145216296096417082_n.jpg
'Applied Physics 184 FX-2'
 Architectural_designs
'BS Applied Physics'
 Classroom
'Colab Notebooks'
'CONSENT_NARAG_MARK JEREMY_G.pdf'
 CONSENT_NARAG_MARKJEREMY_G.pdf
'Dorm bill.png'
'Getting started.pdf'
 IMG_6546.PNG
'IMG_9359 (1).PNG'
 IMG_9359.PNG
'Information Sharing Consent Form - NARAG.pdf'
'Information Sharing Consent Form.pdf'
'[Journal of Cultural Heritage] Discovering artistic influences of painters'
'MS PHYSICS'
'MS Thesis videos'
'NARAG, MARK JEREMY, GACIAS.gdoc'
'NARAG, MARK JEREMY, GACIAS.pdf'
'NARAG, MARK JEREMY.jpg'
 NIP_Narag_MarkJeremy_Consent.pdf
 NIP_Narag_MarkJeremy_Informal.jpg
 NIP_Narag_MarkJeremy_Sablay.jpg
'PEHA 2021 Consent Form (fillable).pdf'
'PHOTO_NARAG_MARK JEREMY_G.jpg'
 PHOTO_NARAG_MARKJEREMY_G.jpg
'Physics 265'
'Physics 301 2S AY2122'
'Physics 305 Data Driven Astronomy'
'Research files'
'Screen Shot 2022-08-31 at 10.46.42.png'
'Screen Shot 2022-09-

In [39]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf


from PIL import Image, ImageOps
Image.MAX_IMAGE_PIXELS = None
import os
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import pandas as pd

from tensorflow import keras

import time
import os
import cv2

In [40]:
def crop(im):
    width, height = im.size
    data = []
    for j in range(0,int(height/n_size)):
        for i in range(0,int(width/n_size)):
            im1 = im.crop((0 + (n_size*i), 0 + (n_size*j), n_size + (n_size*i), n_size + (n_size*j)))
            im1 = np.array(im1)
            im1 = im1.astype(np.float32)
            im1 = im1/255
            data.append(im1)
    return data


In [42]:
gt = Image.open(file_path+ '4 Testing/ground_truth_100dpi.png').convert('L')
img_gt = cv2.imread(file_path+ '4 Testing/ground_truth_100dpi.png')
gt = np.asarray(gt) > 128
gt = ~gt

filename = "nbs_100dpi.png"
directory = file_path + '4 Testing/' + filename
#directory = 'test_output_v'+str(model)+'.png'
dirty = ImageOps.grayscale(Image.open(directory))
dirty = np.array(dirty)
dirty = Image.fromarray(dirty.astype('uint8'))

w_dirty, h_dirty = dirty.size


METRIC = []
channel = ['gray','red','green', 'blue']
for model in range(1,7):
  for color in range(0,4):
      print('Model = ', model, '', str(channel[color]))
      from tensorflow import keras
      autoencoder = keras.models.load_model(file_path + '1 Models/100 DPI/autoencoder_'+str(model)+ '_'+str(channel[color]))
      if model == 1:
          n_size = 32
      if model == 2:
          n_size = 32
      if model == 3:
          n_size = 64
      if model == 4:
          n_size = 64
      if model == 5:
          n_size = 128
      if model == 6:
          n_size = 128
      if model == 7:
          n_size = 256
      if model == 8:
          n_size = 256

      xx = int(w_dirty/n_size)
      final=[]

      for portion in range(0,xx):
          im1 = dirty.crop((n_size*portion, 0, (n_size*portion) + n_size, h_dirty))
          w1, h1 = im1.size
          w = int(w1/n_size)
          h = int(h1/n_size)

          neverbeforeseen = np.array(crop(im1))
          encoded_imgs = autoencoder.encoder(neverbeforeseen).numpy()
          decoded_imgs = autoencoder.decoder(encoded_imgs).numpy()

          col = np.vstack((decoded_imgs[0],decoded_imgs[1]))
          for i in range(2,h):
              col = np.vstack((col,decoded_imgs[i]))

          y = np.where(col > 0.5,1,0) #round the values
          y = (y * 255).astype('uint8')
          if portion == 0:
              final = y
          if portion > 0:
              final = np.hstack((final,y))

      final = np.squeeze(final)
      reconstructed = Image.fromarray(final)
      reconstructed.save(file_path + "5 Tested/output v" + str(model) + '_'+str(channel[color]) + '.png')

      output = Image.open(file_path + "5 Tested/output v" + str(model)  + '_'+str(channel[color]) +  '.png').convert('L')  # convert to grayscale
      img_out = cv2.imread(file_path + "5 Tested/output v" + str(model)  + '_'+str(channel[color]) +  '.png')

      output = np.asarray(output) > 128 # Convert to binary images
      output = ~output

      tp = np.sum(output & gt)
      fp = np.sum(output & ~gt)
      fn = np.sum(~output & gt)
      precision = tp / (tp + fp)
      recall = tp / (tp + fn)
      f1score = (2*precision*recall)/(precision + recall)

      I_1 = output & gt
      U_1 = output | gt
      I = np.sum(output & gt)
      U = np.sum(output | gt)
      i_o_u = I/U

      psnr = cv2.PSNR(img_out, img_gt)

      METRIC.append([model, str(channel[color]), precision,recall,f1score,i_o_u,psnr])
  #COMBINING RGB ------------------------------------------------
  print('Model = ', model, '', str('RGB'))
  output_red = (np.asarray(Image.open(file_path + "5 Tested/output v" + str(model)  + '_'+str(channel[0]) +  '.png').convert('L'))>128)
  output_green = (np.asarray(Image.open(file_path + "5 Tested/output v" + str(model)  + '_'+str(channel[1]) +  '.png').convert('L'))>128)
  output_blue = (np.asarray(Image.open(file_path + "5 Tested/output v" + str(model)  + '_'+str(channel[2]) +  '.png').convert('L'))>128)
  output_rgb = ~output_red | ~output_green | ~output_blue

  reconstructed = Image.fromarray(~output_rgb)
  reconstructed.save(file_path + "5 Tested/output v" + str(model) + '_RGB.png')

  output = Image.open(file_path + "5 Tested/output v" + str(model)  + '_RGB.png').convert('L')  # convert to grayscale
  img_out = cv2.imread(file_path + "5 Tested/output v" + str(model) + '_RGB.png')

  output = np.asarray(output) > 128
  output = ~output

  tp = np.sum(output & gt)
  fp = np.sum(output & ~gt)
  fn = np.sum(~output & gt)
  precision = tp / (tp + fp)
  recall = tp / (tp + fn)
  f1score = (2*precision*recall)/(precision + recall)

  I_1 = output & gt
  O_1 = output | gt
  I = np.sum(output & gt)
  O = np.sum(output | gt)
  i_o_u = I/O

  psnr = cv2.PSNR(img_out, img_gt)

  METRIC.append([model, str('RGB'), precision,recall,f1score,i_o_u,psnr])
  #COMBINING RGB plus grayscale------------------------------------------------
  print('Model = ', model, '', str('RGB+gray'))
  output_gray = (np.asarray(Image.open(file_path + "5 Tested/output v" + str(model) +  '_gray.png').convert('L'))>128)
  output = output_rgb | ~output_gray

  reconstructed = Image.fromarray(~output)
  reconstructed.save(file_path + "5 Tested/output v" + str(model) + '_RGB_GRAY.png')

  output = Image.open(file_path + "5 Tested/output v" + str(model)  + '_RGB_GRAY.png').convert('L')  # convert to grayscale
  img_out = cv2.imread(file_path + "5 Tested/output v" + str(model) + '_RGB_GRAY.png')

  output = np.asarray(output) > 128
  output = ~output

  tp = np.sum(output & gt)
  fp = np.sum(output & ~gt)
  fn = np.sum(~output & gt)
  precision = tp / (tp + fp)
  recall = tp / (tp + fn)
  f1score = (2*precision*recall)/(precision + recall)

  I_1 = output & gt
  O_1 = output | gt
  I = np.sum(output & gt)
  O = np.sum(output | gt)
  i_o_u = I/O

  psnr = cv2.PSNR(img_out, img_gt)

  METRIC.append([model, str('RGB+gray'),precision,recall,f1score,i_o_u,psnr])


METRIC = pd.DataFrame(METRIC)

headers = ['Model','Channel','Precision', 'Recall', 'F1 Score', 'IOU','PSNR' ]
METRIC.columns = headers

METRIC.to_csv(file_path + 'Metric_RGB.csv')
print(METRIC)

Model =  1  gray
Model =  1  red
Model =  1  green
Model =  1  blue
Model =  1  RGB
Model =  1  RGB+gray
Model =  2  gray
Model =  2  red
Model =  2  green
Model =  2  blue
Model =  2  RGB
Model =  2  RGB+gray
Model =  3  gray
Model =  3  red
Model =  3  green
Model =  3  blue
Model =  3  RGB
Model =  3  RGB+gray
Model =  4  gray
Model =  4  red
Model =  4  green
Model =  4  blue
Model =  4  RGB
Model =  4  RGB+gray
Model =  5  gray
Model =  5  red
Model =  5  green
Model =  5  blue
Model =  5  RGB
Model =  5  RGB+gray
Model =  6  gray
Model =  6  red
Model =  6  green
Model =  6  blue
Model =  6  RGB
Model =  6  RGB+gray
    Model   Channel  Precision    Recall  F1 Score       IOU       PSNR
0       1      gray   0.867576  0.650753  0.743683  0.591954  15.279349
1       1       red   0.858124  0.709101  0.776527  0.634691  15.689654
2       1     green   0.866114  0.658176  0.747962  0.597396  15.328133
3       1      blue   0.927917  0.339797  0.497437  0.331059  13.430682
4       1 

In [ ]:
TP = output & gt
TP = Image.fromarray(~TP)
plt.imshow(TP,cmap = 'gray')
TP.save(file_path + "5 Tested/TP.png")

FP = output & ~gt
FP = Image.fromarray(~FP)
plt.imshow(FP,cmap = 'gray')
FP.save(file_path + "5 Tested/FP.png")

FN = ~output & gt
FN = Image.fromarray(~FN)
plt.imshow(FN,cmap = 'gray')
FN.save(file_path + "5 Tested/FN.png")